In [56]:
#prepare data for ai training
import pandas as pd
import pathlib
import os

#load the extracted buildings table
project_root = pathlib.Path(os.getcwd()).parent
data_path = project_root / "buildings_table.csv"
df = pd.read_csv(data_path)

C:\Users\paul_ip\AppData\Local\Temp\ipykernel_6604\3611659020.py:9: DtypeWarning: Columns (6,15,24,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [58]:
def check_entity_overlap(entities, start_idx, end_idx):
    for ent in entities:
        if not (end_idx <= ent[0] or start_idx >= ent[1]):
            return True
    return False

In [ ]:
import re
training_data = []

for idx, row in df.iterrows():
    eng_address = row['eng_full_address']

    entities = []
    pripority_entities = []

    #extract region from the address to avoid overlapping entities due to substring matching
    region = str(row['eng_region'])
    #get the rightmost occurrence of the region in the address
    start_idx = eng_address.rfind(region)
    if start_idx != -1:
        end_idx = start_idx + len(region)
        pripority_entities.append((start_idx, end_idx, 'eng_region'))

    #extract district from the address to avoid overlapping entities due to substring matching
    district = str(row['eng_engdistrict'])
    start_idx = eng_address.rfind(district)
    if start_idx != -1:
        end_idx = start_idx + len(district)
        pripority_entities.append((start_idx, end_idx, 'eng_engdistrict'))

    #handle block number separately to avoid substring matching issues
    if(not pd.isna(row['eng_blockno'])):
        block_str = f'{row["eng_blockno"]} {row["eng_blockdescriptor"]}'
        start_idx = eng_address.find(block_str)
        if start_idx != -1:
            end_idx = start_idx + len(block_str)
            pripority_entities.append((start_idx, end_idx, 'eng_blockno'))

    skip_columns = ['eng_full_address', 'eng_engdistrict', 'eng_blockno', 'eng_blockdescriptor', 'eng_region']
    for col in df.columns:
        if not col.startswith('eng_') or col == 'eng_full_address' or col in skip_columns or row[col] is None:
            continue
        #try to convert building number to int to avoid decimal point issues
        if(col == 'eng_buildingnoto' or col == 'eng_buildingnofrom'):
            try:
                value = str(int(row[col]))
            except:
                value = str(row[col])
        else:
            value = str(row[col])
            
        if value and value != 'nan':
            is_found = False
            for matches in re.finditer(re.escape(value), eng_address):
                start_idx = matches.start()
                end_idx = matches.end()
                if not check_entity_overlap(pripority_entities + entities, start_idx, end_idx):
                    entities.append((start_idx, end_idx, col))
                    is_found = True
                    break
            if not is_found:
                raise ValueError(f'Value "{value}" from column "{col}" not found in address: {eng_address}')

    entities = sorted(entities, key=lambda x: x[1])
    last_end = -1
    last_label = ''
    for start, end, label in entities:
        if start < last_end:
            print(f'Overlapping entities found in address: {eng_address}')
            print(f'{last_label} ({last_end}) overlaps with {label} ({start})')
            print(f'last: {row[last_label]}, current: {row[label]}')
            raise ValueError(f'Overlapping entities found in address: {eng_address}')
        last_end = end
        last_label = label

    training_data.append((row['eng_full_address'], {"entities": entities}))

ValueError: Value "87.0" from column "eng_buildingnoto" not found in address: 85 87 Y FAIRVIEW PARK YUEN LONG FAIRVIEW PARK SECT L 5TH STREET YUEN LONG DISTRICT NT

In [ ]:
#save training data to a file
output_path = project_root / "training_data.pkl"
import pickle
with open(output_path, 'wb') as f:
    pickle.dump(training_data, f)